In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = "prod"


log = structlog.get_logger()

2024-10-16 18:52:55 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="105376883:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-16 18:52:56 [info     ] Loaded 5 items into vault.    
2024-10-16 18:52:57 [info     ] Initialized Clickhouse client.
2024-10-16 18:52:57 [info     ] Query success: blocks in 1.74s 2000 rows
2024-10-16 18:52:58 [info     ] Query success: transactions in 2.36s 14166 rows
2024-10-16 18:52:58 [info     ] Query success: logs in 2.58s 59381 rows


In [3]:
dataframes

{'blocks': shape: (2_000, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 676868   ┆ 30000000  ┆ 0x         ┆ 5              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 1149120  ┆ 30000000  ┆ 0x         ┆ 7              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 3368746  ┆ 30000000  ┆ 0x         ┆ 9              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 417

In [4]:
# # check if our blocks are in the range we want
# log.info(f"Min block number: {dataframes["blocks"]["number"].min()}")
# log.info(f"Max block number: {dataframes["blocks"]["number"].max()}")

# log.info(f"Min block number: {datetime.fromtimestamp( dataframes["blocks"]["timestamp"].min() )}")
# log.info(f"Max block number: {datetime.fromtimestamp( dataframes["blocks"]["timestamp"].max() )}")

2024-10-16 18:52:58 [info     ] Min block number: 105376000   
2024-10-16 18:52:58 [info     ] Max block number: 105377999   
2024-10-16 18:52:58 [info     ] Min block number: 2023-06-09 18:46:17
2024-10-16 18:52:58 [info     ] Max block number: 2023-06-09 19:52:55


In [5]:
dataframes["transactions"]["receipt_l1_fee_scalar"]

receipt_l1_fee_scalar
f64
null
0.684
0.684
0.684
0.684
…
0.684
0.684
0.684


In [6]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [7]:
# list(result.head()["hash"])
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fee,l2_contrib_gas_fees_priority_fee,avg_l2_gas_price_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,17]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0xa4e29db539566c480fbf48ad66a3…",10,"""op""","""2023-06-09""",2,2,0,2,2,0,105377061,105377071,11,10,11,2,554463,554463,0,7416,7416,0,0.0000928869167702470000,0.0000928869167702470000,0.0000000000000000000000,0.0000000001658204040000,0.0000928867509498430000,0.0000000000000000000000,0.0000928867509498440153,0.0000000000299769690000,0.0000000001358434350000,0.00000029906486816,2023-06-09 23:21:39,2023-06-09 23:21:59,20s,1,0.041667,2,2,0
"""0x5511ef434c16d126464238062247…",10,"""op""","""2023-06-09""",7,7,0,7,7,0,105376375,105377732,1358,0,6,7,2101214,2101214,0,52408,52408,0,0.0006474036449318910000,0.0006474036449318910000,0.0000000000000000000000,0.0000000093201616600000,0.0006473943247702310000,0.0000000000000000000000,0.0006473943247702350463,0.0000000093201616600000,0.0000000000000000000000,0.00000443560801517,2023-06-09 22:58:47,2023-06-09 23:44:01,45m 14s,2,0.083333,3,3,0
"""0xbec6606a5c665a176db1300e24d7…",10,"""op""","""2023-06-09""",3,3,0,3,3,0,105376803,105377115,313,432,434,3,1569888,1569888,0,15064,15064,0,0.0002054057770890380000,0.0002054057770890380000,0.0000000000000000000000,0.0000000004725362880000,0.0002054053045527500000,0.0000000000000000000000,0.0002054053045527516193,0.0000000000941932800000,0.0000000003783430080000,0.00000030100000000,2023-06-09 23:13:03,2023-06-09 23:23:27,10m 24s,1,0.041667,1,1,0
"""0x2b7c4f597972feb0db51281d7623…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105376045,105376045,1,8,8,1,390711,390711,0,3596,3596,0,0.0000440181913800020000,0.0000440181913800020000,0.0000000000000000000000,0.0000000000386803890000,0.0000440181526996130000,0.0000000000000000000000,0.0000440181526996137615,0.0000000000203169720000,0.0000000000183634170000,0.00000009900000000,2023-06-09 22:47:47,2023-06-09 22:47:47,0µs,1,0.041667,1,1,0
"""0xd75af65e049392c13fa41b442085…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105377887,105377887,1,0,0,1,240408,240408,0,4528,4528,0,0.0000563644686447520000,0.0000563644686447520000,0.0000000000000000000000,0.0000000001560247920000,0.0000563643126199600000,0.0000000000000000000000,0.0000563643126199607001,0.0000000000129820320000,0.0000000001430427600000,0.00000064900000000,2023-06-09 23:49:11,2023-06-09 23:49:11,0µs,1,0.041667,1,1,0


In [8]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-16 18:52:58 [info     ] Writing 1 partitions to BQ [2023-06-09 00:00:00 ... 2023-06-09 00:00:00]
2024-10-16 18:52:58 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.9Krows 351.1KB to BQ temp.daily_address_summary
